In [2]:
from __future__ import absolute_import, print_function

import tvm
from tvm import te
import numpy as np

tgt_host = "llvm"
tgt = "cpu"

In [2]:
np.random.seed(0)
n = 100
a = np.random.normal(size=n).astype(np.float32)
b = np.random.normal(size=n).astype(np.float32)
c = a + b

In [3]:
def vector_add(a,b,c):
    for i in range(n):
        c = a[i] + b[i]

d = np.empty(shape=n, dtype=np.float32)
vector_add(a,b,d)
np.testing.assert_array_equal(c,d)

/home/lei/.local/lib/python3.8/site-packages/numpy/testing/_private/utils.py:828: RuntimeWarning: overflow encountered in true_divide
  max_rel_error = max(error[nonzero] / abs(y[nonzero]))


AssertionError: 
Arrays are not equal

Mismatched elements: 100 / 100 (100%)
Max absolute difference: 3.1661038e+38
Max relative difference: inf
 x: array([ 3.647203e+00, -9.476018e-01, -2.917470e-01,  3.210290e+00,
        6.944346e-01,  9.663433e-01,  5.364695e-01, -8.988121e-01,
        1.819723e+00,  1.891113e+00,  2.011602e+00,  2.360318e+00,...
 y: array([ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
        7.413770e-31, -6.618781e+13,  4.723853e-37,  4.582806e-41,
        4.762137e-37,  4.582806e-41,  1.899187e-06, -1.755496e-20,...

In [4]:
# Save to the d2ltvm package.
def get_abc(shape, constructor=None):
    """Return random a, b and empty c with the same shape.
    """
    np.random.seed(0)
    a = np.random.normal(size=shape).astype(np.float32)
    b = np.random.normal(size=shape).astype(np.float32)
    c = np.empty_like(a)
    if constructor:
        a, b, c = [constructor(x) for x in (a, b, c)]
    return a, b, c

In [5]:
A = te.placeholder((n,), name="A")
B = te.placeholder((n,), name="B")
C = te.compute(A.shape, lambda i: A[i] + B[i], name="C")
(A.dtype, A.shape),(C.dtype, C.shape)

(('float32', [100]), ('float32', [100]))

In [6]:
type(A.op),type(C.op)

(tvm.te.tensor.PlaceholderOp, tvm.te.tensor.ComputeOp)

In [7]:
s = te.create_schedule(C.op)
type(s),type(s[C]),type(s[C.op])

(tvm.te.schedule.Schedule, tvm.te.schedule.Stage, tvm.te.schedule.Stage)

In [8]:
tvm.lower(s,[A,B,C],name="main",simple_mode=True)

IRModuleNode( {GlobalVar(main): PrimFunc([A, B, C]) attrs={"global_symbol": "main", "tir.noalias": (bool)1} {
  for (i, 0, 100) {
    C[i] = (A[i] + B[i])
  }
}
})

In [9]:
target = "llvm"
module = tvm.build(s, [A,B,C], target)
type(module)

tvm.runtime.module.Module

In [11]:
x = np.ones(2)
y = tvm.nd.array(x)
type(y), y.asnumpy()

(tvm.runtime.ndarray.NDArray, array([1., 1.]))

In [13]:
a, b, c = get_abc(100, tvm.nd.array)

In [21]:
module(a,b,c)

In [22]:
a.asnumpy() + b.asnumpy() == c.asnumpy()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [25]:
try:
    a,b,c = get_abc(200, tvm.nd.array) # tvm will checkout the input shape 
    module(a,b,c)
except tvm.TVMError as e:
    print(e)

Traceback (most recent call last):
  [bt] (2) /home/lei/tvm/build/libtvm.so(TVMFuncCall+0x63) [0x7fbfd7765763]
  [bt] (1) /home/lei/tvm/build/libtvm.so(+0x1243821) [0x7fbfd7775821]
  [bt] (0) /home/lei/tvm/build/libtvm.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x80) [0x7fbfd6ba8310]
  File "/home/lei/tvm/src/runtime/library_module.cc", line 78
TVMError: Check failed: ret == 0 (-1 vs. 0) : Assert fail: (100 == int32(arg0.shape[0])), Argument arg0.shape[0] has an unsatisfied constraint


In [26]:
type(a),type(b),type(c)

(tvm.runtime.ndarray.NDArray,
 tvm.runtime.ndarray.NDArray,
 tvm.runtime.ndarray.NDArray)

In [27]:
a.shape,b.shape,c.shape

((200,), (200,), (200,))

In [28]:
a.dtype,b.dtype,c.dtype # should be same with ABC

('float32', 'float32', 'float32')

In [30]:
try:
    a,b,c = get_abc(200, tvm.nd.array) # tvm will checkout the input shape 
    a = tvm.nd.array(a.asnumpy().astype(np.float64))
    module(a,b,c)
except tvm.TVMError as e:
    print(e)

Traceback (most recent call last):
  [bt] (2) /home/lei/tvm/build/libtvm.so(TVMFuncCall+0x63) [0x7fbfd7765763]
  [bt] (1) /home/lei/tvm/build/libtvm.so(+0x1243821) [0x7fbfd7775821]
  [bt] (0) /home/lei/tvm/build/libtvm.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x80) [0x7fbfd6ba8310]
  File "/home/lei/tvm/src/runtime/library_module.cc", line 78
TVMError: Check failed: ret == 0 (-1 vs. 0) : Assert fail: (((tir.tvm_struct_get(arg0, 0, 5) == (uint8)2) && (tir.tvm_struct_get(arg0, 0, 6) == (uint8)32)) && (tir.tvm_struct_get(arg0, 0, 7) == (uint16)1)), arg0.dtype is expected to be float32


In [31]:
# save and load module
module_fname = 'vector_add.tar'
module.export_library(module_fname)

In [32]:
!ls

opt_gemm.ipynb		 tvm_import.ipynb	      vector_add.tar
relay_quick_start.ipynb  tvm_tensor_expression.ipynb


In [33]:
loaded_module = tvm.runtime.load_module(module_fname)

In [35]:
try:
    a,b,c = get_abc(100, tvm.nd.array) # tvm will checkout the input shape 
    loaded_module(a,b,c)
except tvm.TVMError as e:
    print(e)

## Reduction Operation

In [12]:
a = np.arange(12)
a.resize((3,4))
sum_a = a.sum(axis=0)
a, sum_a

(array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]),
 array([12, 15, 18, 21]))